### **Purpose:** Demonstrates basic text generation using the **Amazon Bedrock Runtime API**.  
This code sends a simple prompt to a foundation model (**Titan Text Express**) and retrieves the generated response.  
It shows how to directly invoke a model with a single user prompt using `invoke_model` (or `converse`) without maintaining conversation history.


In [15]:
import boto3
import json

# Create Bedrock Runtime client
bedrock = boto3.client("bedrock-runtime", region_name="us-east-1")

response = bedrock.converse(
    modelId="amazon.titan-text-express-v1",
    messages=[
        {"role": "user", "content": [{"text": "Summarize what DevOps is in one line."}]}
    ],
)

print(json.dumps(response["output"]["message"]["content"][0]["text"], indent=2))


"\n\nDevOps is a methodology that combines software development and IT operations to shorten the systems development life cycle while delivering features, enhancements, and fixes quicker.\n"


### **Purpose:** Demonstrates **Retrieval-Augmented Generation (RAG)** using the **Amazon Bedrock Agent Runtime API**.  
The code retrieves relevant context from a connected **Knowledge Base** (stored in an **S3 bucket**) and generates a contextual response using the **Claude model**.  
It illustrates how Bedrock combines **retrieval (from stored documents)** and **generation (via LLMs)** to answer domain-specific queries.


In [ ]:
import boto3
import json

# Create Bedrock Agent Runtime client
agent_runtime = boto3.client("bedrock-agent-runtime", region_name="us-east-1")

response = agent_runtime.retrieve_and_generate(
    input={"text": "How do I restart the staging environment?"},
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            "knowledgeBaseId": "YOUR_KNOWLEDGE_BASE_ID_HERE",  # Update with your KB ID
            "modelArn": "arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-haiku-20240307-v1:0"
        }
    }
)

print("\nModel Output:\n")
print(json.dumps(response["output"]["text"], indent=2))



Model Output:

"To restart the staging environment, you should run the script `restart-staging.sh` on the staging server."


### **Purpose:** Demonstrates how to build a **multi-turn conversational chatbot** using the **Amazon Bedrock Converse API**.  
This example defines a **system prompt** (role/persona) and maintains **conversation history** across multiple user messages.  
It shows how to manage dialogue context and model responses using parameters like `temperature`, `topP`, and `maxTokens` for creative yet coherent outputs.


In [17]:
import boto3
import json

# -----------------------------------------------------------
# 1️⃣ Create a Bedrock Runtime client
# -----------------------------------------------------------
# Used to send requests to Amazon Bedrock models.
# Make sure AWS credentials are configured and region is correct.
bedrock = boto3.client("bedrock-runtime", region_name="us-east-1")

# The Claude model we’ll use for text generation
MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"


# -----------------------------------------------------------
# 2️⃣ Define the system prompt (model’s role/personality)
# -----------------------------------------------------------
# This tells the model *who it is* and *how to behave*.
SYSTEM_PROMPT = [
    {
        "text": (
            "You are an app that creates playlists for a radio station "
            "that plays rock and pop music. Only return song names and artists."
        )
    }
]


# -----------------------------------------------------------
# 3️⃣ Set inference parameters (control creativity)
# -----------------------------------------------------------
# temperature → randomness / creativity level (0 = focused, 1 = creative)
# topP        → limits how random tokens are chosen (nucleus sampling)
# maxTokens   → caps how long the response can be
INFERENCE_CONFIG = {
    "temperature": 0.7,
    "topP": 0.9,
    "maxTokens": 512
}


# -----------------------------------------------------------
# 4️⃣ Helper function: send request to model
# -----------------------------------------------------------
def ask_model(messages):
    """
    Sends conversation history (messages) to the model and
    returns only the generated text response.
    """
    response = bedrock.converse(
        modelId=MODEL_ID,
        system=SYSTEM_PROMPT,
        messages=messages,
        inferenceConfig=INFERENCE_CONFIG,
    )
    return response["output"]["message"]["content"][0]["text"]


# -----------------------------------------------------------
# 5️⃣ Main conversation flow
# -----------------------------------------------------------
def main():
    # Step 1: user starts conversation
    messages = [
        {"role": "user", "content": [{"text": "Create a list of three rock songs."}]}
    ]

    # Step 2: get first model reply
    first_reply = ask_model(messages)
    print("\n--- First Response ---")
    print(first_reply)

    # Step 3: add model’s reply back into conversation history
    messages.append({"role": "assistant", "content": [{"text": first_reply}]})

    # Step 4: ask a follow-up question (with context)
    messages.append({
        "role": "user",
        "content": [{"text": "Make sure the songs are by artists from the United Kingdom."}]
    })

    # Step 5: get contextual (second) response
    second_reply = ask_model(messages)
    print("\n--- Second Response ---")
    print(second_reply)


# -----------------------------------------------------------
# 6️⃣ Entry point (run main)
# -----------------------------------------------------------
if __name__ == "__main__":
    main()



--- First Response ---
Here are three rock songs:

1. "Smells Like Teen Spirit" by Nirvana
2. "Sweet Child O' Mine" by Guns N' Roses
3. "Bohemian Rhapsody" by Queen

--- Second Response ---
Okay, here are three rock songs by artists from the United Kingdom:

1. "Satisfaction" by The Rolling Stones
2. "Paranoid" by Black Sabbath
3. "Layla" by Derek and the Dominos


===============================================
### 🎯 PURPOSE:
This code demonstrates how to **create and use a Prompt Template** 
with Amazon Bedrock's Converse API.

### 🔑 KEY CONCEPT:
Templates are like reusable functions - you define them once with 
placeholders (like {{event_request}}), then "call" them multiple 
times by passing different values through `promptVariables`.

### 📋 WHAT THIS CODE DOES:
1. Creates a prompt template for screening venue booking requests
2. The template has a placeholder: {{event_request}}
3. Calls the template by passing actual booking text via promptVariables
4. Model processes the template and returns YES/NO answers

### 🔄 WORKFLOW:
Step 1: bedrock-agent.create_prompt()  → Creates template with {{variable}}
Step 2: bedrock.converse()             → Calls template with actual values
        - modelId = prompt_arn         → Points to the template
        - promptVariables = {...}      → Injects values into {{variable}}

### 💡 ANALOGY:
Think of it like a form letter:
- Template: "Dear {{name}}, welcome to {{company}}..."  (Step 1)
- Calling:  name="Alice", company="AWS"                 (Step 2)
- Result:   "Dear Alice, welcome to AWS..."

===============================================


In [1]:

import boto3

# ==============================
# Initialize AWS Clients
# ==============================
# bedrock_agent: For creating/managing templates
# bedrock: For running the model (inference)
bedrock_agent = boto3.client("bedrock-agent", region_name="us-east-1")
bedrock = boto3.client("bedrock-runtime", region_name="us-east-1")

# Model to use
MODEL_ID_NOVA = "us.amazon.nova-micro-v1:0"

# ==============================
# STEP 1: Create the Prompt Template
# ==============================
# This is like defining a function with parameters
# Template has: {{event_request}} as a placeholder variable

try:
    response = bedrock_agent.create_prompt(
        name="Event-Booking-Screener-1",
        description="Checks if a booking request meets venue requirements",
        variants=[
            {
                "name": "Variant1",
                "modelId": MODEL_ID_NOVA,
                "templateType": "CHAT",
                "inferenceConfiguration": {
                    "text": {
                        "temperature": 0.4  # Low = more consistent answers
                    }
                },
                "templateConfiguration": {
                    "chat": {
                        # System prompt: Tells model its role and instructions
                        "system": [
                            {
                                "text": """You are helping a venue manager quickly screen booking requests.
                                Read the full message and answer YES or NO to the following.
                                Be strict, and if you don't have enough info, just say NO.
                                
                                    A) Does the request mention fewer than 150 guests?
                                    B) Is the requested time within 10am to 11pm?
                                    C) Does the event avoid restricted activities like live animals or open flames?
                                    D) Is there any indication that ALCOHOL service would be handled by a licensed vendor?
                                    E) Is there enough information provided to follow up with the requester?"""
                            }
                        ],
                        # User message with PLACEHOLDER variable
                        "messages": [
                            {
                                "role": "user",
                                "content": [
                                    {
                                        # {{event_request}} will be replaced when we call the template
                                        "text": "Booking request: {{event_request}}"
                                    }
                                ]
                            }
                        ],
                        # Declare what variables this template expects
                        "inputVariables": [
                            {"name": "event_request"}
                        ]
                    }
                }
            }
        ]
    )
    
    print("Created!")
    prompt_arn = response.get("arn")
    
except bedrock_agent.exceptions.ConflictException as e:
    # If template already exists, just reuse it
    print("Already exists!")
    response = bedrock_agent.list_prompts()
    prompt = next((prompt for prompt in response['promptSummaries'] 
                   if prompt['name'] == "Event-Booking-Screener-1"), None)
    prompt_arn = prompt['arn']

# Print the template's unique identifier
print(prompt_arn)

# ==============================
# STEP 2: CALL THE TEMPLATE (The Magic Part!)
# ==============================
# This is like calling a function: my_function(parameter="value")
# 
# HOW IT WORKS:
# - modelId=prompt_arn          → Instead of model ID, use TEMPLATE ARN
# - promptVariables={...}       → This is where we "pass values" to {{event_request}}
#
# Bedrock will:
# 1. Load the template
# 2. Replace {{event_request}} with the actual text below
# 3. Send the complete prompt to the model
# 4. Return the model's response

response = bedrock.converse(
    modelId=prompt_arn,  # ← CRITICAL: Use template ARN, not model ID!
    promptVariables={     # ← This "calls" the template with actual values
        "event_request": {
            "text": """Hi, I'm looking to book your event space for a birthday party. We're expecting around 90 guests,
                and planning to start around 6pm and wrap up by 10:30pm. It will just have
                a DJ and dancing. We'd like to serve wine and beer — my cousin has a catering company with a liquor
                license who would handle that. Let me know if this date is available and what next steps would be. Thanks!
                """
        }
    }
)

# ==============================
# STEP 3: Display the Response
# ==============================
# Model will answer YES or NO to each screening question
print("\n" + "="*60)
print("MODEL RESPONSE:")
print("="*60)
print(response['output']['message']['content'][0]['text'])

Created!
arn:aws:bedrock:us-east-1:976193224059:prompt/U9MWH81BUW

MODEL RESPONSE:
A) YES - The request mentions fewer than 150 guests (90 guests).
B) YES - The requested time is from 6pm to 10:30pm, which falls within 10am to 11pm (considering the wrap-up time).
C) YES - The event avoids restricted activities like live animals or open flames (it will just have a DJ and dancing).
D) YES - There is an indication that alcohol service would be handled by a licensed vendor (cousin's catering company with a liquor license).
E) YES - There is enough information provided to follow up with the requester (date, number of guests, time frame, type of event, and details on alcohol service).

Based on the information provided, the answer is YES for all criteria.


In [2]:
# ==============================
# 🧪 BONUS: How to Reuse Template with Different Requests
# ==============================
print("\n" + "="*60)
print("TESTING WITH ANOTHER BOOKING REQUEST:")
print("="*60)

# Same template, different values - this is the power of templates!
response2 = bedrock.converse(
    modelId=prompt_arn,  # Reusing same template
    promptVariables={
        "event_request": {
            "text": """Need to book space for 200 people at 2am. 
We'll bring our pet tigers for entertainment. BYOB situation."""
        }
    }
)

print(response2['output']['message']['content'][0]['text'])


TESTING WITH ANOTHER BOOKING REQUEST:
NO

A) Does the request mention fewer than 150 guests?
NO, the request mentions 200 people.

B) Is the requested time within 10am to 11pm?
NO, the requested time is 2am, which is outside the acceptable time frame.

C) Does the event avoid restricted activities like live animals or open flames?
NO, the request mentions bringing pet tigers, which is a restricted activity.

D) Is there any indication that ALCOHOL service would be handled by a licensed vendor?
NO, the request mentions BYOB (Bring Your Own Bottled), but it does not specify that alcohol will be handled by a licensed vendor.

E) Is there enough information provided to follow up with the requester?
NO, the request lacks sufficient details and includes several major restrictions violations.
